In [ ]:
### imports

# external modules
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

import plotting.plottools as plottools
importlib.reload(plottools)
import tools.omstools as omstools
importlib.reload(omstools)

In [ ]:
# define eras and files to run over

eras = [
  #'Run2024A-v1',
  'Run2024B-v1',
  'Run2024C-v1',
  'Run2024D-v1',
  'Run2024E-v1',
  'Run2024E-v2',
  'Run2024F-v1',
  'Run2024G-v1',
  'Run2024H-v1',
  'Run2024I-v1',
  'Run2024I-v2',
  #'Run2024J-v1',
  #'Run2025A-v1',
  #'Run2025A-v2',
  'Run2025B-v1',
  'Run2025C-v1',
  'Run2025C-v2',
  'Run2025D-v1',
  'Run2025E-v1',
  'Run2025F-v1',
  'Run2025F-v2',
  'Run2025G-v1',
]

omsfiles = {}
hltfiles = {}
for era in eras: omsfiles[era] = f'omsdata/omsdata_{era}.json'
for era in eras: hltfiles[era] = f'omsdata/hltrate_{era}.json'

In [ ]:
# make plots of luminosity and pileup
# note: the unit of luminosity per lumisection is inverse pb, as retrieved from looking at OMS tables.

# settings
plot_run_numbers = False

for era in eras:
    
    # load the file
    with open(omsfiles[era], 'r') as f:
        info = json.load(f)
        
    # get indices where the run number changes
    runs = np.array(info['run_number']).astype(int)
    run_info = {}
    for run in set(runs):
        indices = np.asarray(np.array(runs)==run).nonzero()[0]
        start_idx = indices[0]
        nlumis = len(indices)
        run_info[run] = {'start_idx': start_idx, 'nlumis': nlumis}
        
    # make plot of luminosity
    fig, ax = plt.subplots(figsize=(18,6))
    fig, ax = plottools.plot_hists([info['delivered_lumi_per_lumisection'], info['recorded_lumi_per_lumisection']],
               fig=fig, ax=ax,
               colorlist=['b','g'],
               labellist=['Delivered lumi', 'Recorded lumi'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='Luminosity (/pb)', yaxtitlesize=15,
               title='Luminosity for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None,
               bkgcolor=info['physics_flag'], bkgcmap='spring', bkgcolorbar=True, bkgtitle='Physics flag set')
    
    # add lines for run numbers
    if plot_run_numbers:
        yscale = (ax.get_ylim()[1] - ax.get_ylim()[0])
        modidx = 0
        for run, this_run_info in run_info.items():
            if this_run_info['nlumis'] < 500: continue # to avoid drawing too many lines
            ax.axvline(this_run_info['start_idx'], color='grey', linestyle='--')
            height = ax.get_ylim()[1] - (yscale*0.1)*(modidx+1)
            txt = ax.text(this_run_info['start_idx'] + 25, height, run, va='top', fontsize=12)
            txt.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='white'))
            modidx += 1
            modidx = modidx % 3
    
    # make plot of pileup
    fig, ax = plt.subplots(figsize=(18,6))
    fig, ax = plottools.plot_hists([info['pileup']],
               fig=fig, ax=ax,
               colorlist=['b'],
               labellist=['Pileup'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='Pileup', yaxtitlesize=15,
               title='Pileup for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None,
               bkgcolor=info['physics_flag'], bkgcmap='spring', bkgcolorbar=True, bkgtitle='Physics flag set')
    
    # add lines for run numbers
    if plot_run_numbers:
        yscale = (ax.get_ylim()[1] - ax.get_ylim()[0])
        modidx = 0
        for run, this_run_info in run_info.items():
            if this_run_info['nlumis'] < 500: continue # to avoid drawing too many lines
            ax.axvline(this_run_info['start_idx'], color='grey', linestyle='--')
            height = ax.get_ylim()[1] - (yscale*0.1)*(modidx+1)
            txt = ax.text(this_run_info['start_idx'] + 25, height, run, va='top', fontsize=12)
            txt.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='white'))
            modidx += 1
            modidx = modidx % 3

In [ ]:
# make plots of trigger rates

# settings
plot_run_numbers = False

for era in eras:
    
    # open the file and read the run numbers
    if era not in hltfiles.keys(): continue
    if not os.path.exists(hltfiles[era]): continue
    with open(hltfiles[era], 'r') as f:
        data = json.load(f)
    runs = sorted(list(data.keys()))
    print(f'Found {len(runs)} runs for era {era}.')
    
    # loop over the runs and read the rates
    run_start_ids = [0]
    run_info = {}
    triggerdata = {}
    for run in runs:
        thisdata = data[run]
        triggers = list(thisdata.keys())
        if len(triggers)==0: continue
        nlumis = len(thisdata[triggers[0]]['rate'])
        start_idx = run_start_ids[-1]
        stop_idx = start_idx + nlumis
        xax = np.arange(start_idx, stop_idx)
        for trigger in triggers:
            if trigger in triggerdata.keys(): triggerdata[trigger].append((xax, thisdata[trigger]['rate']))
            else: triggerdata[trigger] = [(xax, thisdata[trigger]['rate'])]
        run_info[run] = {'start_idx': start_idx, 'nlumis': nlumis}
        run_start_ids.append(stop_idx)
    
    # flatten the ranges for the same trigger
    for trigger, batches in triggerdata.items():
        xax = np.concatenate([b[0] for b in batches])
        rate = np.concatenate([b[1] for b in batches])
        triggerdata[trigger] = (xax, rate)
    
    # make the plot
    fig, ax = plt.subplots(figsize=(18,6))
    cids = np.linspace(0, 1, num=len(triggerdata))
    cmap = plt.get_cmap('cool')
    for idx, (trigger, data) in enumerate(triggerdata.items()):
        xax = data[0]
        rate = data[1]
        ax.plot(xax, rate, label=trigger, color=cmap(cids[idx]))
    ax.set_yscale('log')
    ax.set_xlabel('Lumisection', fontsize=15)
    ax.set_ylabel('HLT rate (Hz)', fontsize=15)
    ax.set_title(f'HLT rates for selected triggers in {era}', fontsize=15)
    ax.legend(fontsize=12)
    ax.grid(which='both')
    
    # add lines and run numbers
    if plot_run_numbers:
        yscale = (ax.get_ylim()[1]/ax.get_ylim()[0])
        modidx = 0
        for run, info in run_info.items():
            if info['nlumis'] < 100: continue # to avoid drawing too many lines
            ax.axvline(info['start_idx'], color='grey', linestyle='--')
            height = ax.get_ylim()[1] / (yscale*5e-5)**(modidx+1)
            txt = ax.text(info['start_idx'] + 25, height, run, va='top', fontsize=12)
            txt.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='white'))
            modidx += 1
            modidx = modidx % 3

In [ ]:
# investigate particular cases

importlib.reload(omstools)

era = 'Run2024B-v1'
run = 378981
ls = 667

# get OMS info
with open(omsfiles[era], 'r') as f:
    omsinfo = json.load(f)
pileup = omstools.find_oms_attr_for_lumisections([run], [ls], omsinfo, 'pileup', verbose=True)
print(pileup)

# get trigger rate
with open(hltfiles[era], 'r') as f:
    hltinfo = json.load(f)
hltrate = omstools.find_hlt_rate_for_lumisections([run], [ls], hltinfo, 'HLT_ZeroBias_v*', verbose=True)
print(hltrate)